# Step 3. Train the model (LSTM)

Using the data created from Step 1, here we are going to build a simple benchmark model (simple neural network) to evaluate the stock return predictability.

We find the model in Step 2 is barely the same as Random walk. Let's try whether a more complicated model, LSTM can be helpful.

In [1]:
# data 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import os

%matplotlib inline

In [2]:
# sagemaker
import boto3
import sagemaker
from sagemaker import get_execution_role

In [3]:
# SageMaker session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# default S3 bucket
bucket = sagemaker_session.default_bucket()

In [4]:
data_dir = 'data' # the folder we will use for storing data
train_name = 'train.csv'

In [5]:
# specify where to upload in S3
prefix = 'stock-price-predictor'

# upload to S3
test_location = sagemaker_session.upload_data(os.path.join(data_dir, 'test.csv'), key_prefix=prefix)
train_location = sagemaker_session.upload_data(os.path.join(data_dir, 'train.csv'), key_prefix=prefix)

print(train_location)
print(test_location)

s3://sagemaker-ap-northeast-1-165829362107/stock-price-predictor/train.csv
s3://sagemaker-ap-northeast-1-165829362107/stock-price-predictor/test.csv


In [6]:
!pygmentize source_lstm/model.py

import torch
import torch.nn as nn
import torch.nn.functional as F

# Here we define our model as a class
class SimpleLSTM(nn.Module):
    def __init__(self, input_dim, feature_dim, hidden_dim, output_dim):
        super().__init__()

        self.feature_dim = feature_dim
        self.fc1 = nn.Linear(input_dim, self.feature_dim)

        self.hidden_layer_size = hidden_dim
        self.lstm = nn.LSTM(self.feature_dim, hidden_dim)
        self.hidden_cell = (torch.zeros(1, 1, self.hidden_layer_size),
                            torch.zeros(1, 1, self.hidden_layer_size))

        self.drop = nn.Dropout(0.1)
        self.fc2 = nn.Linear(hidden_dim, output_dim)


    def forward(self, input_seq):

        out = input_seq.view(len(input_seq), 1, -1)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.drop(out)

        out, self.hidden_cell = self.lstm(out, self.hidden_cell)
        out = out.view(len(out), -1)
        out = self.drop(out)

        predictions = self.f

In [7]:
!pygmentize source_lstm/train.py

from __future__ import print_function  # future proof
import argparse
import sys
import os
import json

import pandas as pd

# pytorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data

# import model
from model import SimpleLSTM

def model_fn(model_dir):
    print("Loading model.")

    # First, load the parameters used to create the model.
    model_info = {}
    model_info_path = os.path.join(model_dir, 'model_info.pth')
    with open(model_info_path, 'rb') as f:
        model_info = torch.load(f)

    print("model_info: {}".format(model_info))

    # Determine the device and construct the model.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = SimpleLSTM(model_info['input_dim'],
                       model_info['feature_dim'],
                       model_info['hidden_dim'],
                       model_info['output_dim'])

    # Load the stored model parameters.
    model_path = os.path.join(model_dir, 'mod

## Train the Model

In [8]:
# import a PyTorch wrapper
from sagemaker.pytorch import PyTorch

# specify an output path
# prefix is specified above
output_path = 's3://{}/{}'.format(bucket, prefix)

# instantiate a pytorch estimator
estimator = PyTorch(entry_point='train.py',
                    source_dir='source_lstm', # this should be just "source" for your code
                    role=role,
                    framework_version='1.3.1',
                    train_instance_count=1,
                    train_instance_type='ml.c4.8xlarge',
                    output_path=output_path,
                    sagemaker_session=sagemaker_session,
                    hyperparameters={
                        'input_dim': 46,  # num of features
                        'hidden_dim': 3,
                        'feature_dim': 5,
                        'output_dim': 1,
                        'epochs': 5, # could change to higher
                        'validation_split': 0.2,
                    })

In [9]:
s3_input_train = sagemaker.s3_input(s3_data=train_location, content_type='csv')

In [10]:
# First, make sure to import the relevant objects used to construct the tuner
from sagemaker.tuner import IntegerParameter, ContinuousParameter, HyperparameterTuner

hyperparameter_tuner = HyperparameterTuner(estimator = estimator, # The estimator object to use as the basis for the training jobs.
                                           objective_metric_name = 'validation:rmse', # The metric used to compare trained models.
                                           metric_definitions=[{'Name': 'validation:rmse',
                                                                'Regex': 'validation:rmse=([0-9\\.]+)'}],
                                           objective_type = 'Minimize', # Whether we wish to minimize or maximize the metric.
                                           max_jobs = 30, # The total number of models to train
                                           max_parallel_jobs = 6, # The number of models to train in parallel
                                           hyperparameter_ranges = {
                                               'hidden_dim': IntegerParameter(5, 20),
                                               'feature_dim': IntegerParameter(5, 20),
                                               'validation_split': ContinuousParameter(0.1, 0.3),
                                               'epochs': IntegerParameter(5, 10),
                                           },
                                          )

In [11]:
hyperparameter_tuner.fit({'train': s3_input_train})

In [ ]:
hyperparameter_tuner.wait()

....

In [ ]:
estimator_attached = sagemaker.estimator.Estimator.attach(hyperparameter_tuner.best_training_job())

In [ ]:
estimator_attached.set_hyperparameters(validation_split=0)
estimator_attached.hyperparameters()

In [ ]:
# Fit the validation set into the model to get the final model

estimator_attached.fit({'train': s3_input_train})

## Deploy the Model

In [ ]:
# importing PyTorchModel
from sagemaker.pytorch import PyTorchModel

# Create a model from the trained estimator data
# And point to the prediction script
model = PyTorchModel(model_data=estimator_attached.model_data,
                     role = role,
                     framework_version='1.3.1',
                     entry_point='predict.py',
                     source_dir='source_lstm')

In [ ]:
%%time
# deploy and create a predictor
predictor = model.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

In [ ]:
input_data = np.array(pd.read_csv("data/test.csv"))

X_test = input_data[:, 1:]
y_test = input_data[:, 0]

y_pred = predictor.predict(X_test)

In [ ]:
plt.plot(y_pred)

In [ ]:
y_pred = y_pred.reshape(y_pred.shape[0])
y_test = y_test.reshape(y_pred.shape[0])

accuracy = ( np.logical_or( np.logical_and(y_pred >= 0, y_test >= 0),
                            np.logical_and(y_pred < 0, y_test < 0) ) ).sum() / len(y_test)
print("Accuracy =", accuracy)

In [ ]:
plt.figure()
plt.plot(y_test, y_pred, ls=' ', marker='.')
plt.show()

**Conclusion:**

By tests that are not listed here, if we try to forecast the entire test set (forecasting 4 years based on this simple model), then the performance is really poor: the accuaracy is only about 48% to 52%, and our model is always predicting negative numbers, so the accuracy actually only depends on how many returns are really negative.

One of the underlying problems, is that LSTM does not learn from the predicted output against the real output aggregately. In that way, the actual returns, essential for the data, did not get integrated with our model. Therefore, it's unlikely that our model still preverse long-term dependency. The model must learn from the output of each iteration of forecasting.

So our model may only be useful for short-term forecasting. Here I presented my methods when only forecasting 22 days, which is roughly the trading dates of a month. As we can see, my model predicts negative signs for all the returns in `GE`, and the actual accuracy is about 60%. This is a sign that my model says that this stock is not a good investment, and therefore it's always predicting negative signs. In fact, the actual returns have 60% of the times being negative.

It is therefore possible that the performance of my model has enhanced. However, more further steps need to be done for better analysis. For example, more high-frequency and up-to-date data, and better implement of model that can learn real-time, so the model can be useful in predicting long-term and it can keep learning.

In [46]:
# Accepts a predictor endpoint as input
# And deletes the endpoint by name
def delete_endpoint(predictor):
        try:
            boto3.client('sagemaker').delete_endpoint(EndpointName=predictor.endpoint)
            print('Deleted {}'.format(predictor.endpoint))
        except:
            print('Already deleted: {}'.format(predictor.endpoint))

In [51]:
# delete the predictor endpoint 
delete_endpoint(predictor)

Deleted pytorch-inference-2020-02-20-09-15-31-886
